In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import pandas as pd
import time 
import os
import pathlib
import datetime
import matplotlib.pyplot as plt
from torch import optim
from torch.autograd import Variable 
from pathlib import Path
from random import randint
import torch.nn as nn
import torch.nn.functional as F
from statistics import mode, mean
import matplotlib.pyplot as plt

directory = r'C:\Users\savva\OneDrive\Desktop\npy'
patient_ID = ['2zbyXzYNKPwiPtjaA2L64o.npy','3hY7Mp7u9YPo1xMARSxLhc.npy','4h1dAuzg9rdrhyojwxUS26.npy']
colums1 = ['time','end time']
colums2 = ['time','end time','Reading 1']
colums3 = ['time','end time','Reading 1','Reading 2']
door_key = 0
switch = 0
pir_key = 0
hidden_size = 2
hidden_size
def Data_Loader(patient):
    sensors_lst = []
    column_names = []
    for subdirectory in os.scandir(directory):
        lock = False
        for patients in os.scandir(subdirectory):
            if str(pathlib.Path(patients)).endswith(patient_ID[0]):
                if lock == False:
                    column_names.append(str(os.path.basename(subdirectory)))
                    lock == True                                              
                sensors = np.load(pathlib.Path(patients))
                sensors_lst.append(sensors)
    sensors_pd = pd.DataFrame([sensors_lst],columns=column_names)
    dataframe_lst = []
    df = pd.DataFrame()
    sensor_num = 2
    Sensor_ID = column_names[sensor_num]
    print(Sensor_ID)
    for i in range(len(sensors_pd[Sensor_ID].loc[0])):
        dataframe_lst.append(sensors_pd[Sensor_ID].loc[0][i])
    if len(sensors_pd[Sensor_ID].loc[0][0]) == 2: 
        df = pd.DataFrame(dataframe_lst,columns = colums1)
        df = df.drop("end time", axis=1)
    elif len(sensors_pd[Sensor_ID].loc[0][0]) == 3:    
        df = pd.DataFrame(dataframe_lst,columns = colums2)
        df = df.drop("end time", axis=1)
    elif len(sensors_pd[Sensor_ID].loc[0][0]) == 4:    
        df = pd.DataFrame(dataframe_lst,columns = colums3)
        df = df.drop("end time", axis=1)
    df['labels'] = 0
    time = df['time']
    anomaly_lst = []
    col_name = df.columns
    col_len = len(df.columns)
    delta_t = []
    n = df.columns[0]
    df.drop(n, axis = 1, inplace = True)
    for i in range(0,len(df)-1):
        delta = abs(df.iloc[i,0] - df.iloc[i+1,0])
        delta_t.append(delta)
        i = i+1
    df[n] = pd.Series(delta_t)
    df.dropna(axis=1,inplace=False)
    return df[0:10020],time

def create_anomaly(time,n,id):
    global pir_time
    global pir_key
    int = 0.0 # there are 20 locations
    if pir_key == 0:
        pir_time = time
        s = [str(integer) for integer in pir_time]
        a_string = "".join(s)
        pir_time = float(a_string)
        pir_key = 1
    if pir_key == 1:
        pir_time = 60.0
        return [int] + [0] + [pir_time] 

def spike_anomaly(time,n,id): # for ambient temp
    global pir_time
    global pir_key
    spikeint = 200.0 
    room_id = 15.0
    if pir_key == 0:
        pir_time = time
        s = [str(integer) for integer in pir_time]
        a_string = "".join(s)
        pir_time = float(a_string)
        pir_key = 1
    if pir_key == 1:
        pir_time = 60.0
        return [room_id] + [spikeint] + [0] + [pir_time] 

def variance_anomaly(time,n,id): # for ambient temp
    global pir_time
    global pir_key
    variancen = randint(0.0,200.0)
    variancen = variancen/10
    if pir_key == 0:
        pir_time = time
        s = [str(integer) for integer in pir_time]
        a_string = "".join(s)
        pir_time = float(a_string)
        pir_key = 1
    if pir_key == 1:
        pir_time = 30.0
        return [id] + [variancen] + [0] + [pir_time] 


def anomaly_insertion(n,type,dataframe,id):
    df = dataframe
    anomaly_lst = []
    col_name = df.columns
    col_len = len(df.columns)
    start_t = min(dataframe['time'])
    end_t = max(dataframe['time'])
    for i in range(n):
        random_timestamp = random.randint(start_t,end_t)
        if type == 'create':
            zp_anomaly_lst = create_anomaly([random_timestamp],col_len,id)
        if type == 'spike':
            zp_anomaly_lst = spike_anomaly([random_timestamp],col_len,id)
        if type == 'variance':
            zp_anomaly_lst = variance_anomaly([random_timestamp],col_len,id)           
        anomaly_lst.append(zp_anomaly_lst)

    anomaly_df = pd.DataFrame(anomaly_lst,columns=col_name)
    concat_df = pd.concat([dataframe,anomaly_df],ignore_index=True)
    concat_df = concat_df.reset_index(drop=True)
    return  concat_df

dataT,time1 = Data_Loader(patient_ID[0])
df1,time2 = Data_Loader(patient_ID[0])
time1 = time1[0:10020]
time2 = time2[0:10040]
# dataV = anomaly_insertion(300,'create',df1,id = None)
#dataV = anomaly_insertion(1,'spike',df1,id = None)
dataV1 = anomaly_insertion(10,'variance',df1,5)
dataV = anomaly_insertion(10,'variance',dataV1,7)
dataT = dataT.astype('float32')
dataV = dataV.astype('float32')

train_data = [dataT]
train_data = pd.concat(train_data)
valid_data = dataV

X_trainD = train_data.loc[:, train_data.columns != 'labels']
X_valD = valid_data.loc[:, valid_data.columns != 'labels']
Y_trainD = train_data.loc[:,'labels']
Y_valD = valid_data.loc[:,'labels']

X_train = torch.tensor(X_trainD.values).float()
X_val =torch.tensor(X_valD.values).float()
Y_train = torch.tensor(Y_trainD.values).float()
Y_val = torch.tensor(Y_valD.values).float()

Y_tensor = Y_train.clone().detach()
new_shape = (len(Y_tensor), 1)
Y_train = Y_tensor.view(new_shape)

Y_tensor = Y_val.clone().detach()
new_shape = (len(Y_tensor), 1)
Y_val = Y_tensor.view(new_shape)

torch.flatten(X_train)
torch.flatten(Y_train)
torch.flatten(X_val)
torch.flatten(Y_val)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 512)        
        self.fc4 = nn.Linear(512, 2)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr = 3e-5)
criterion = nn.MSELoss()
batch_size = 1
n_epochs = 1
permutation_train = torch.randperm(X_train.size()[0])
permutation_val = torch.randperm(X_val.size()[0])
anomalies = []
loss_history = []
history = []

def Train_AD():
    for epoch in range(n_epochs):
        accuracy = 0
        accuracy_history = []
        timer_plot = []
        for i in range(0,len(train_data), batch_size):
            optimizer.zero_grad()
            indices = permutation_train[i:i+batch_size]
            batch_x, batch_y = X_train[indices], Y_train[indices]
            outputs = model.forward(batch_x)
            loss = criterion(outputs,batch_y)
            loss_history.append(loss.item())
            loss.backward()
            optimizer.step()
            print(f"-> Training :   "+str(i)+"/"+str(X_train.size()[0]), end = "\r")  
        threshold = max(loss_history) - mode(loss_history)
        for a in range(0,X_val.size()[0], batch_size):
            start_time = time.time() 
            indicesV = permutation_val[a:a+batch_size]
            batch_x_val, batch_y_val = X_val[indicesV], Y_val[indicesV]
            output = model.forward(batch_x_val)
            pred = torch.max(output,1)[1]
            loss = criterion(output,batch_y_val)
            history.append(loss.item())
            if loss.item() >= threshold:
                anomalies.append(batch_x_val)
            time_elapsed = time.time() - start_time
            timer_plot.append(1000*time_elapsed/len(X_val))
            print(f"-> Testing :     "+str(a)+"/"+str(X_val.size()[0]), end = "\r")
        print(f"Finished                                                                                         ")
    return anomalies
anomalies = Train_AD()

In [39]:
print(anomalies)

[tensor([[ 5.0000, 14.6000, 40.0000]]), tensor([[ 5.0000, 14.8000, 40.0000]]), tensor([[ 7.,  1., 40.]]), tensor([[ 7.0000, 13.5000, 40.0000]]), tensor([[ 5.,  3., 40.]]), tensor([[ 7.0000, 19.4000, 40.0000]]), tensor([[ 1., 22.,  1.]]), tensor([[18.0000, 23.6000, 15.0000]]), tensor([[ 5.0000,  4.2000, 40.0000]]), tensor([[ 2.0000, 25.5000,  0.0000]]), tensor([[ 1.0000, 21.1000,  1.0000]]), tensor([[18.0000, 24.7000,  0.0000]]), tensor([[ 5.,  9., 40.]]), tensor([[ 7.0000,  8.8000, 40.0000]]), tensor([[ 5.0000,  3.5000, 40.0000]]), tensor([[ 7.0000,  5.2000, 40.0000]]), tensor([[ 7.0000, 14.8000, 40.0000]]), tensor([[ 1.0000, 22.3000,  1.0000]]), tensor([[ 5.0000, 19.8000, 40.0000]]), tensor([[18.0000, 24.8000,  0.0000]]), tensor([[ 7.0000, 18.5000, 40.0000]]), tensor([[ 7.0000,  7.4000, 40.0000]]), tensor([[ 1.0000, 22.1000,  1.0000]]), tensor([[18.0000, 24.6000,  0.0000]]), tensor([[ 1.0000, 21.3000,  1.0000]]), tensor([[ 7.0000,  5.2000, 40.0000]]), tensor([[ 3.0000, 26.3000,  1.000

In [ ]:

import pandas as pd

dates1 = pd.to_datetime(time1, utc=True, unit='s')
dates2 = pd.to_datetime(time2, utc=True, unit='s')
lstDateTime1 = dates1.to_list()
lstDateTime2 = dates2.to_list()
plt.figure(figsize=(16, 3))
plt.rcParams.update({'font.size': 12})
plt.title('Training and Validation loss for half a month, with 20 variance anomalies')
plt.plot(lstDateTime1,loss_history,label = 'training loss')
plt.plot(lstDateTime2,history,label = 'validation loss')
plt.legend()
plt.ylim([0,0.6])



In [ ]:

counter = 0
for i in range(len(anomalies)):
    x = anomalies[i].numpy()
    if x[0][2]==40.0:
        counter = counter+1

print(counter)
print(counter/20)

